In [1]:
# Reference
# https://linuxhint.com/python_xml_to_dictionary/

In [97]:
import os
import xmltodict
import math
import random
from copy import deepcopy

In [2]:
base = './data/ori'

In [7]:
def convert2coco(base):
    
    # Open an xml file and convert it into the dictionary format
    def xml_to_dict(base, filename):
        with open(os.path.join(base, filename), "r") as xml_obj:
            my_dict = xmltodict.parse(xml_obj.read())
            xml_obj.close()
        return my_dict
    
    # Get filenames of xml files
    filenames = [filename for filename in os.listdir(base) if os.path.splitext(filename)[-1] == '.xml']
    
    # Initialize the annotations dictionary
    annotations = {
        "type": "instances",
        "images": [],
        "categories": [],
        "annotations": []
    }
    
    img2annots = {}

    cls_dict = {}
       
    # Get all classes in the dataset
    for filename in filenames:
        ann  = xml_to_dict(base, filename)['annotation']
        for obj in ann['object']:
            cls_name = obj['name']
            if cls_name not in cls_dict:
                cls_dict[cls_name] = len(cls_dict) + 1
    
    img_id = 1 # for img counting
    ann_id = 1 # for annotation counting
    for filename in filenames:
        img2annots[filename] = {
            'data': None,
            'annotations': [],
            'num_objects': None
        }
        
        # Get annotations of an xml file
        ann  = xml_to_dict(base, filename)['annotation']
            
        for ann_obj in ann['object']:
            bbx = ann_obj['bndbox']
            xmin, ymin = int(bbx['xmin']), int(bbx['ymin'])
            xmax, ymax = int(bbx['xmax']), int(bbx['ymax'])
            dx = xmax - xmin
            dy = ymax - ymin
            
            annot = {
                "id": ann_id,
                "bbox": [xmin, ymin, dx, dy],
                "image_id": img_id,
                "category_id": cls_dict[ann_obj['name']],
                "segmentation": [],
                "area": dx*dy,
                "iscrowd": 0
            }
            annotations["annotations"].append(annot)
            ann_id = ann_id + 1
            
            img2annots[filename]['annotations'].append(annot)
        
        size = ann['size']
        image = {
            "file_name": ann['filename'],
            "height":size['height'] ,
            "width": size['width'],
            "id": img_id
        }
        annotations["images"].append(image)
        img_id = img_id + 1
        
        img2annots[filename]['data'] = image       

    for cls_name, cls_id in cls_dict.items():
        annotations["categories"].append({
            "supercategory": "none",
            "name": cls_name,
            "id": cls_id
        })
        
    for filename, val in img2annots.items():
        img2annots[filename]['num_objects'] = {cls_id: 0 for _, cls_id in cls_dict.items()}
        for obj in val['annotations']:
            category_id = obj['category_id']
            img2annots[filename]['num_objects'][category_id] = img2annots[filename]['num_objects'][category_id] + 1
        
    return {
        'type': annotations['type'],
        'categories': annotations['categories'],
        'img2annots': img2annots
    }

In [8]:
annotations = convert2coco(base)

In [154]:
def train_test_split(img2annots, split_dictionary, max_iter=10000):
    total = sum([val for _, val in split_dictionary.items()])
    split_dict = {key: val/total for key, val in split_dictionary.items()}
    
    # Get the number of objects
    categories = img2annots[list(img2annots.keys())[0]]['num_objects']
    total_objects = {key: 0 for key, _ in categories.items()}
    for key, val in img2annots.items():
        for cat_id, cat_n in val['num_objects'].items():
            total_objects[cat_id] = total_objects[cat_id] + cat_n
    
    # Get the spit_size for every set
    total_img = len(img2annots.keys())
    split_size_img = {}
    for key1, val1 in split_dict.items():
        split_size_img[key1] = math.ceil(val1*total_img)
            
    # Get the index_mapping for each set
    split_img_dict = {}
    start_idx = 0
    for key, val in split_size_img.items():
        split_img_dict[key] = [start_idx, min(start_idx + val, total_img)]
        start_idx = start_idx + val
        
    # Calculate the percentage of objects w.r.t to total objects
    def calculate_object(data_dict, total_objects):
        count = {key: 0 for key, _ in total_objects.items()}
        for key, val in data_dict.items():
            for ann in val['annotations']:
                category_id = ann['category_id']
                count[category_id] = count[category_id] + 1
        for key, val in total_objects.items():
            count[key] = count[key] / val
        return count
            
    # Optimization
    img_name = list(img2annots.keys())
    obj_counts = {}
    best_error = 1.
    for i in range(max_iter):
        random.shuffle(img_name)
        
        for key, val in split_img_dict.items():
            obj_dict = {name: img2annots[name] for name in img_name[val[0]:val[1]]}
            obj_counts[key] = calculate_object(obj_dict, total_objects)
            
        error = 0
        for key1, val1 in split_dictionary.items():
            for key2, val2 in obj_counts[key1].items():
                error = error + (val1-val2)**2
        
        if error < best_error:
            best_error = deepcopy(error)
            best_img_name_seq = deepcopy(img_name)
            print(best_error)
            print(obj_counts)
    
    

In [155]:
split_dictionary = {
    'train': 0.60,
    'val': 0.20,
    'test': 0.20
}

train_test_split(annotations['img2annots'], split_dictionary)

0.0012908747888896812
{'train': {1: 0.5871405891691657, 2: 0.6098635708838234}, 'val': {1: 0.20964896807197037, 2: 0.21599864418269638}, 'test': {1: 0.203210442758864, 2: 0.1741377849334802}}
0.0005490599942986379
{'train': {1: 0.591550538013759, 2: 0.6030844843657317}, 'val': {1: 0.2054154171811607, 2: 0.183882721803237}, 'test': {1: 0.20303404480508025, 2: 0.21303279383103127}}
0.00024071960048803763
{'train': {1: 0.6030164050097019, 2: 0.6059655961359207}, 'val': {1: 0.19500793790792026, 2: 0.20566053724260655}, 'test': {1: 0.20197565708237786, 2: 0.18837386662147276}}
0.00017567324242240073
{'train': {1: 0.5949902981125419, 2: 0.5984238623845437}, 'val': {1: 0.19439054506967718, 2: 0.20193203965765613}, 'test': {1: 0.2106191568177809, 2: 0.1996440979578002}}
7.376561133843132e-05
{'train': {1: 0.6007232316105133, 2: 0.605033471739683}, 'val': {1: 0.19641912153819016, 2: 0.20015252944665707}, 'test': {1: 0.20285764685129654, 2: 0.19481399881365985}}
3.449570874224321e-05
{'train': {